In [2]:
import torch
topic=10
data_RNA=torch.load("/data/nelkazwi/lymphoma_data/rna_lymphoma.txt")
dev = "cuda:0"
device = torch.device(dev)

theta=torch.load("/scratch/nelkazwi/lymphoma_data/"+str(topic)+"_topics/theta_"+str(topic)+".txt")
lam=torch.load("/scratch/nelkazwi/lymphoma_data/"+str(topic)+"_topics/lam_"+str(topic)+".txt")


theta=theta[50:,:,:]
lam=lam[50:,:,:]

def WAIC_g (lam_,theta_,data_RNA_,c_):
    N=lam_.shape[0]
    T=lam_.shape[1]
    G=lam_.shape[2]
    C=theta_.shape[1]
    
    L=torch.zeros([C,G],dtype=torch.double,device=device)
    V=torch.zeros([C,G],device=device)
    V_2=torch.zeros([C,G],device=device)
    
    for n in torch.arange(0,N):
     theta_tmp=theta_[n,:,:].to(device)
     lam_tmp=lam_[n,:,:].to(device)
     theta_tmp=theta_tmp.type(torch.cuda.DoubleTensor)
     lam_tmp=lam_tmp.type(torch.cuda.DoubleTensor)
     for i in torch.arange(1,c_.shape[0]): 
           data_RNA_1=data_RNA_[c_[i-1]:c_[i],:].to(device)
           m=torch.distributions.poisson.Poisson(lam_tmp.reshape([T,1,G]))
           z_=torch.mul(torch.exp(m.log_prob(data_RNA_1)),(theta_tmp[c_[i-1]:c_[i],:].T)[:,:,None])

           
           z_=torch.sum(z_,axis=0)
           L[c_[i-1]:c_[i],:]+=z_
           
           V[c_[i-1]:c_[i],:]+=torch.pow(torch.log(z_),2)
           
           V_2[c_[i-1]:c_[i],:]+=torch.log(z_)
   
    WAIC=-2*(torch.sum(torch.log(L/N))-(torch.sum(V)-torch.sum(torch.pow(V_2,2)/N))/(N-1))
    return WAIC



n_cells=theta.shape[1]
batch_size=500
n_batches=n_cells+batch_size-(n_cells%batch_size)
c=torch.arange(0,n_batches,batch_size)
c=torch.hstack((c,torch.tensor(n_cells)))
c=c.to(device)
L_g=WAIC_g (lam,theta,data_RNA,c)
torch.save(L_g,"/scratch/nelkazwi/lymphoma_data/"+str(topic)+"_topics/WAIC_g"+str(topic)+".txt")